In [2]:
# Задание 1 из 3
# Дополните код скрипта и запустите его для получения ответа (в виде пока что пустого списка) от сервиса рекомендаций.

import requests

recommendations_url = "http://127.0.0.1:8000/recommendations"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 0}

resp = requests.post(recommendations_url, headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

{'recs': []}


In [12]:
# Шаг 2. Подключение готовых рекомендаций

# Добавим пользы в наш сервис. Для этого сделаем так, чтобы при его запуске загружались уже готовые рекомендации, 
# а затем и отдавались при вызове /recommendations. Для удобства основной инструментарий разместим в отдельном классе, 
# который приведён ниже в готовом виде:

import logging as logger
import pandas as pd

class Recommendations:

    def __init__(self):

        self._recs = {"personal": None, "default": None}
        self._stats = {
            "request_personal_count": 0,
            "request_default_count": 0,
        }

    def load(self, type, path, **kwargs):
        """
        Загружает рекомендации из файла
        """

        logger.info(f"Loading recommendations, type: {type}")
        self._recs[type] = pd.read_parquet(path, **kwargs)
        if type == "personal":
            self._recs[type] = self._recs[type].set_index("user_id")
        logger.info(f"Loaded")

    def get(self, user_id: int, k: int=100):
        """
        Возвращает список рекомендаций для пользователя
        """
        try:
            recs = self._recs["personal"].loc[user_id]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_personal_count"] += 1
        except KeyError:
            recs = self._recs["default"]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_default_count"] += 1
        except:
            logger.error("No recommendations found")
            recs = []

        return recs

    def stats(self):

        logger.info("Stats for recommendations")
        for name, value in self._stats.items():
            logger.info(f"{name:<30} {value} ")

In [13]:
content_recommendations = pd.read_parquet("./candidates/inference/content_recommendations.parquet")
content_recommendations.head()

,user_id,item_id,score
0,1000000,1,0.933434
1,1000000,2,0.925806
2,1000000,3,0.920225
3,1000000,5,0.918026
4,1000000,6,0.916345


In [19]:
als_recommendations = pd.read_parquet("./candidates/inference/als_recommendations.parquet")
als_recommendations.head()

,user_id,item_id,score
0,1000000,3,0.990940
1,1000000,15881,0.896617
2,1000000,5,0.864405
3,1000000,6,0.822256
4,1000000,2,0.774098


In [21]:
# Задание 2 из 3
# Дополните код и вызовите запущенный сервис рекомендаций, чтобы получить от него пустой (пока) список.

rec_store = Recommendations()

rec_store.load(
    "personal",
    "./candidates/inference/als_recommendations.parquet",
    columns=["user_id", "item_id", "score"],
)
rec_store.load(
    "default",
    "./candidates/inference/content_recommendations.parquet",
    columns=["item_id", "score"],
)

rec_store.get(user_id=100, k=5) 

[1, 2, 3, 5, 6]

In [22]:
rec_store.get(user_id=1049126, k=5)

[1885, 10210, 2657, 5297, 6185]

In [25]:
import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1353637, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

{'recs': []}


In [ ]:
# Правильный ответ — {'recs': [28187230, 27161156, 5]}. 
# Это словарь со списком рекомендаций. Если у вас получилось другое значение, проверьте, что вы верно указали значения user_id и k.